Primero de todo se tienen que descargar los datos, que se extraen de la siguiente URL: https://www.kaggle.com/datasets/bryanpark/spanish-single-speaker-speech-dataset?resource=download. En Kaggle hay detalles sobre el dataset. Más tarde, nos quedaremos para este proyecto con la carpeta 19demarzo, una de las 3 que contiene muestras de audio. Además, con los audios se descarga su transcripción. Tendremos que quedarnos sólo con la transcripción referente a la carpeta 19demarzo. 

Más tarde, tenemos que hacer unas pequeñas modificaciones:
 - Se tiene que transformar el txt en 3 csv: metadata_train.csv, metadata_test.csv y metadata_dev.csv
 - Del csv, se quitará la columna final relativa al tiempo, pues se calcula después
 - Se modifica la ruta del audio
 - Se eliminan las filas nulas

Después de estas transformaciones y limpieza de nulos, el dataset estaría listo para usarse.
La estructura de carpetas sería la siguiente:
- datasetCastellanoReducido
-       |-19demarzo 

-           |-19demarzo_0000.wav
-           |-...
-           |-19demarzo_3272.wav
-       |-metadata_train.csv
-       |-metadata_test.csv
-       |-metadata_dev.csv

Este cuaderno de Python funciona para datasets que tengan una carpeta con los ficheros de audio y un fichero de transcript.txt, donde están las transcripciones. Las transcripciones tienen el siguiente formato: 
19demarzo/19demarzo_0000.wav|Durante nuestra conversación advertí que la multitud aumentaba, apretándose más.|Durante nuestra conversación advertí que la multitud aumentaba, apretándose más.|5.88

EXTRACCIÓN DE LOS DATOS

In [1]:
#Se crea el directorio en donde se alojará el dataset
!mkdir /home/irene/datasets/datasetFinetuning

In [3]:
#Se cambia al directorio creado. Hay que asegurarse de que se está en este directorio para bajar el dataset en éste y no en otro lugar.
import os
os.chdir("/home/irene/datasets/datasetEsMapa152Finetuning/esmapa152")

In [3]:
#Aquí se debe poner en /root/.kaggle un fichero llamado kaggle.json
#Además, se crean las siguientes variables (username y kaggle_key), que se cogen de dicho fichero.
#Por último, se descarga el dataset 
!export KAGGLE_USERNAME=irenepenas
!export KAGGLE_KEY=c268b1a61cb47e8e6b774b403dce4698
!kaggle datasets download bryanpark/spanish-single-speaker-speech-dataset

100%|█████████████████████████████████████▉| 6.86G/6.86G [02:55<00:00, 44.9MB/s]
100%|██████████████████████████████████████| 6.86G/6.86G [02:56<00:00, 41.9MB/s]


In [4]:
!unzip spanish-single-speaker-speech-dataset.zip

Archive:  spanish-single-speaker-speech-dataset.zip
  inflating: 19demarzo/19demarzo_0000.wav  
  inflating: 19demarzo/19demarzo_0001.wav  
  inflating: 19demarzo/19demarzo_0002.wav  
  inflating: 19demarzo/19demarzo_0003.wav  
  inflating: 19demarzo/19demarzo_0004.wav  
  inflating: 19demarzo/19demarzo_0005.wav  
  inflating: 19demarzo/19demarzo_0006.wav  
  inflating: 19demarzo/19demarzo_0007.wav  
  inflating: 19demarzo/19demarzo_0008.wav  
  inflating: 19demarzo/19demarzo_0009.wav  
  inflating: 19demarzo/19demarzo_0010.wav  
  inflating: 19demarzo/19demarzo_0011.wav  
  inflating: 19demarzo/19demarzo_0012.wav  
  inflating: 19demarzo/19demarzo_0013.wav  
  inflating: 19demarzo/19demarzo_0014.wav  
  inflating: 19demarzo/19demarzo_0015.wav  
  inflating: 19demarzo/19demarzo_0016.wav  
  inflating: 19demarzo/19demarzo_0017.wav  
  inflating: 19demarzo/19demarzo_0018.wav  
  inflating: 19demarzo/19demarzo_0019.wav  
  inflating: 19demarzo/19demarzo_0020.wav  
  inflating: 19demarzo/1

In [5]:
#Nos quedamos sólo con el dataset 19demarzo y con la transcripción
!rm -r bailen
!rm -r batalla_arapiles
!rm -r es
!rm spanish-single-speaker-speech-dataset.zip

In [4]:
#Quitamos las filas que no sean del 19demarzo
input_file = "transcript.txt"
output_file = "filtered_transcript.txt"
# prefix = "19demarzo/"
prefix = "esmapa152/"


with open(input_file, "r") as file:
    lines = file.readlines()

filtered_lines = [line for line in lines if line.startswith(prefix) and "||" not in line]

with open(output_file, "w") as file:
    file.writelines(filtered_lines)

print("Filtered transcript file has been created successfully.")


Filtered transcript file has been created successfully.


In [4]:
#Hacemos algunas transformaciones y eliminamos nulos
# Open the input file for reading
with open('filtered_transcript.txt', 'r') as input_file:
    # Create a new file for writing the modified contents
    with open('modified_transcript.txt', 'w') as output_file:
        # Process each line in the input file
        for line in input_file:
            # Split the line into parts using the '|' delimiter
            parts = line.strip().split('|')
            
            # Extract the desired components and join them with appropriate formatting
            new_line = '/'.join(parts[0].split('/')[1:]).split('.')[0] + '|' + '|'.join(parts[1:3]) + '\n'
            
            # Write the modified line to the output file
            output_file.write(new_line)


In [8]:
#Ahora tenemos que crear a partir de transcript.txt, 3 csv: metadata_train.csv, metadata_test.csv y metadata_dev.csv
import random

# Set the desired percentages for train, test, and dev
train_percent = 0.95
test_percent = 0.025
dev_percent = 0.025

# Open the modified input file for reading
with open('modified_transcript.txt', 'r') as input_file:
    # Read all lines from the input file
    lines = input_file.readlines()

    # # Shuffle the lines randomly
    # random.shuffle(lines)

    # Calculate the number of lines for each set
    total_lines = len(lines)
    train_lines = int(total_lines * train_percent)
    test_lines = int(total_lines * test_percent)
    dev_lines = total_lines - train_lines - test_lines

    # Create the train, test, and dev sets
    train_set = lines[:train_lines]
    test_set = lines[train_lines:train_lines + test_lines]
    dev_set = lines[train_lines + test_lines:]

    # Write the train set to metadata_train.csv
    with open('metadata_train.csv', 'w') as train_file:
        train_file.writelines(train_set)

    # Write the test set to metadata_test.csv
    with open('metadata_test.csv', 'w') as test_file:
        test_file.writelines(test_set)

    # Write the dev set to metadata_dev.csv
    with open('metadata_dev.csv', 'w') as dev_file:
        dev_file.writelines(dev_set)


In [9]:
!rm modified_transcript.txt
!rm filtered_transcript.txt
!rm transcript.txt